Resnet Implementation

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
import pandas as pd
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
# from util import split_dataset

In [22]:
# load and preprocess data
data = pd.read_csv('fer2013.csv')
width, height = 48, 48
datapoints = data['pixels'].tolist()
faces = []
for pixel_sequence in datapoints:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(width, height)
    faces.append(face.astype('float32'))
faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)
emotions = data['emotion'].values

print(faces.shape)
print(emotions)

(35887, 48, 48, 1)
[0 0 2 ... 0 3 2]


In [35]:
# split data into training, validation, and test sets
# Determine the minimum number of samples across all emotion classes
class_counts = np.bincount(emotions)
min_samples = np.min(class_counts)

# Undersample the classes to match the minimum number of samples
undersampler = RandomUnderSampler(sampling_strategy={i: min_samples for i in range(len(class_counts))}, random_state=42)
X_undersampled, y_undersampled = undersampler.fit_resample(faces.reshape(len(faces), -1), emotions)
X_undersampled = X_undersampled.reshape(-1, 1, 48, 48)  # Reshape back to the original shape

# Split the undersampled data into training, testing, and evaluation sets
train_ratio = 0.8
test_ratio = 0.1
eval_ratio = 0.1

# First, split the data into training and remaining sets
X_train, X_rem, y_train, y_rem = train_test_split(X_undersampled, y_undersampled, test_size=1-train_ratio, stratify=y_undersampled, random_state=42)

# Then, split the remaining data into testing and evaluation sets
test_ratio_adjusted = test_ratio / (test_ratio + eval_ratio)
X_test, X_eval, y_test, y_eval = train_test_split(X_rem, y_rem, test_size=test_ratio_adjusted, stratify=y_rem, random_state=42)

# Oversample the minority classes in the training set
oversampler = RandomOverSampler(sampling_strategy='not majority', random_state=42)
X_train_oversampled, y_train_oversampled = oversampler.fit_resample(X_train.reshape(len(X_train), -1), y_train)
X_train_oversampled = X_train_oversampled.reshape(-1, 1, 48, 48)  # Reshape back to the original shape

In [24]:
# define data transformations
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [25]:
# data loaders!
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train_oversampled), torch.tensor(y_train_oversampled))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(torch.tensor(X_test), torch.tensor(y_test))
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

eval_dataset = torch.utils.data.TensorDataset(torch.tensor(X_eval), torch.tensor(y_eval))
eval_loader = DataLoader(eval_dataset, batch_size=64, shuffle=False)

In [37]:
# define the model
model = models.resnet50(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
num_features = model.fc.in_features
num_classes = 7
model.fc = nn.Linear(num_features, num_classes)

# move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/Users/kadijaismail/miniconda3/envs/EmotionRecML/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kadijaismail/miniconda3/envs/EmotionRecML/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [40]:
def run_epoch(loader, model, criterion, optimizer, device, train=False):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += inputs.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects.double() / total_samples

    return epoch_loss, epoch_acc

def accuracy(loader, model):
    model.eval()
    running_corrects = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            running_corrects += torch.sum(preds == labels.data)
            total_samples += inputs.size(0)

    acc = running_corrects.double() / total_samples

    return acc


In [41]:
# Training loop
max_epochs = 10
eps = 1e-7
iters = 0
train_losses = []

while iters < max_epochs: ## feel free to change the way it loops i got lazy and this is what it is
    train_loss, train_acc = run_epoch(train_loader, model, criterion, optimizer, device, train=True)
    eval_acc = accuracy(eval_loader, model)
    convergence = abs(train_losses[-1] - train_loss) < eps
    ## holy shit this takes forever on a single epoch, im gonna load it up on my desktop tmrw morning
    print(f'Epoch {iters+1}/{max_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Eval Acc: {eval_acc:.4f}')
    train_losses.append(train_loss)
    if convergence:
        print(f'Converged after {iters+1} epochs')
        break
    else:
        iters += 1

# Test the model
test_acc = accuracy(test_loader, model)
print(f'Test Acc: {test_acc:.4f}')

# Save the model
torch.save(model.state_dict(), 'resnet50.pth')
    

Epoch 1/10, Train Loss: 1.7810, Train Acc: 0.3320, Eval Acc: 0.3081
Epoch 2/10, Train Loss: 1.5352, Train Acc: 0.4354, Eval Acc: 0.3159
Epoch 3/10, Train Loss: 1.4004, Train Acc: 0.4948, Eval Acc: 0.3211
Epoch 4/10, Train Loss: 1.2075, Train Acc: 0.5714, Eval Acc: 0.3211
Epoch 5/10, Train Loss: 0.9984, Train Acc: 0.6491, Eval Acc: 0.2742
Epoch 6/10, Train Loss: 0.9762, Train Acc: 0.6601, Eval Acc: 0.3473
Epoch 7/10, Train Loss: 0.6850, Train Acc: 0.7655, Eval Acc: 0.3211
Epoch 8/10, Train Loss: 0.5895, Train Acc: 0.7971, Eval Acc: 0.3185
Epoch 9/10, Train Loss: 0.4765, Train Acc: 0.8330, Eval Acc: 0.3055
Epoch 10/10, Train Loss: 0.4175, Train Acc: 0.8545, Eval Acc: 0.3081
Test Acc: 0.3394
